# Notebook 2 : Pandas avancé

In [1]:
import pandas as pd

## Tennis

Nous considérons les données des résultats des matchs de tennis masculin des tournois de Roland Garros et Wimbledon en 2013. La liste des variables et leur signification se trouvent sur [cette page](https://archive.ics.uci.edu/dataset/300/tennis+major+tournament+match+statistics) dans la section *Additional Variable Information*.

1. Commencer par charger le jeu de données relatif au tournoi de Roland Garros dans un dataframe `rg` à partir du fichier `rolandgarros2013.csv`.

In [4]:
rg = pd.read_csv("data/rolandgarros2013.csv")
rg.head()

,Player1,Player2,Round,Result,FNL.1,FNL.2,FSP.1,FSW.1,SSP.1,SSW.1,...,BPC.2,BPW.2,NPA.2,NPW.2,TPW.2,ST1.2,ST2.2,ST3.2,ST4.2,ST5.2
0,Pablo Carreno-Busta,Roger Federer,1,0,0,3,62,27,38,11,...,7,7,14,18,88,6,6,6.0,NaN,NaN
1,Somdev Devvarman,Daniel Munoz-De La Nava,1,1,3,0,62,54,38,22,...,1,16,22,25,106,3,3,5.0,NaN,NaN
2,Tobias Kamke,Paolo Lorenzi,1,1,3,2,62,53,38,15,...,10,18,19,27,139,3,3,6.0,6.0,3.0
3,Julien Benneteau,Ricardas Berankis,1,1,3,1,72,87,28,19,...,4,13,33,43,149,6,3,7.0,6.0,NaN
4,Lukas Lacko,Sam Querrey,1,0,0,3,52,31,48,22,...,4,7,12,13,93,6,6,6.0,NaN,NaN


2. Afficher les noms des demi-finalistes.

In [5]:
print(
    rg[rg["Round"] == 6][["Player1", "Player2"]]
)

            Player1             Player2
122    David Ferrer  Jo-Wilfried Tsonga
123  Novak Djokovic        Rafael Nadal


3. Calculer le nombre moyen d'aces par match dans le tournoi.

In [6]:
print(
    rg["ACE.1"].agg(["mean"])+rg["ACE.2"].agg(["mean"])
)

mean    12.688
dtype: float64


4. Combien y a-t-il eu d'aces par match en moyenne à chaque niveau du tournoi ?

In [42]:
print(
    (rg["ACE.1"]+rg["ACE.2"])
    .groupby(rg["Round"])
    .mean()
)

Round
1    13.476190
2    13.193548
3    12.562500
4     9.125000
5     7.000000
6    10.000000
7     6.000000
dtype: float64


5. Filtrer les matchs pour lesquels au moins une des variables `DBF.1` et `DBF.2` est manquante.

In [21]:
print(
    rg[pd.isna(rg["DBF.1"])|pd.isna(rg["DBF.2"])][["Player1", "Player2","DBF.1","DBF.2"]]
)

             Player1        Player2  DBF.1  DBF.2
56    Simone Bolelli    Yen-Hsun Lu    NaN    NaN
63  Somdev Devvarman  Roger Federer    NaN    NaN


6. Remplacer les valeurs manquantes de `DBF.1` par zéro avec la méthode `loc`.

In [23]:
rg.loc[rg["DBF.1"].isna(), "DBF.1"] = 0
print(rg[rg["DBF.2"].isna()][["Player1", "Player2","DBF.1","DBF.2"]])

             Player1        Player2  DBF.1  DBF.2
56    Simone Bolelli    Yen-Hsun Lu    0.0    NaN
63  Somdev Devvarman  Roger Federer    0.0    NaN


7. Remplacer les valeurs manquantes de `DBF.2` par zéro avec la méthode `fillna`.

In [32]:
rg["DBF.2"].fillna(0, inplace=True)
print(rg.iloc[[56,63]][["Player1", "Player2","DBF.1","DBF.2"]])

             Player1        Player2  DBF.1  DBF.2
56    Simone Bolelli    Yen-Hsun Lu    0.0    0.0
63  Somdev Devvarman  Roger Federer    0.0    0.0


8. Extraire la liste des participants à partir des colonnes `Player1` et `Player2`. Une façon de faire consiste à utiliser `concat` et la méthode `drop_duplicates` pour obtenir le résultat sous la forme d'une série et de la convertir en dataframe avec la méthode `to_frame`.

In [51]:
# players = pd.concat([rg["Player1"], rg["Player2"]])
# print(players)
# players.drop_duplicates(inplace=True)
# print(players)
# playersdf = players.to_frame(name="Player")
playersdf = pd.concat([rg["Player1"], rg["Player2"]]).drop_duplicates().to_frame(name="Player")
print(playersdf)

                 Player
0   Pablo Carreno-Busta
1      Somdev Devvarman
2          Tobias Kamke
3      Julien Benneteau
4           Lukas Lacko
..                  ...
56          Yen-Hsun Lu
59      Grigor Dimitrov
61          Guido Pella
62         David Goffin
87     Janko Tipsarevic

[127 rows x 1 columns]


9. Écrire une fonction `n_match` qui prend une chaîne de caractères `joueur` en entrée et retourne le nombre de matchs disputés par le joueur.

In [64]:
def n_match(joueur):
    """
    Compte le nombre de matchs joués par un joueur
    """
    return len(rg[rg["Player1"] == joueur]) + len(rg[rg["Player2"] == joueur])
print(n_match("Novak Djokovic"))

6


10. Utiliser les deux question précédentes et la méthode `apply` pour compter le nombre de matchs que chaque participant a disputé et ordonner le résultat par ordre décroissant.

In [65]:
playersdf["Total_matches"] = playersdf["Player"].apply(n_match)
print(playersdf.sort_values("Total_matches", ascending=False))

                  Player  Total_matches
47          Rafael Nadal              7
122         David Ferrer              7
62        Novak Djokovic              6
15    Jo-Wilfried Tsonga              6
113        Tommy Robredo              5
..                   ...            ...
5            Denis Kudla              1
19     Marcel Granollers              1
51        Carlos Berlocq              1
10           James Blake              1
0    Pablo Carreno-Busta              1

[127 rows x 2 columns]


11. Charger maintenant le jeu de données relatif au tournoi de Wimbledon dans un dataframe `wb` à partir du fichier `wimbledon2013.csv`.

In [66]:
wb = pd.read_csv("data/wimbledon2013.csv")
wb.head()

,Player1,Player2,Round,Result,FNL.1,FNL.2,FSP.1,FSW.1,SSP.1,SSW.1,...,BPC.2,BPW.2,NPA.2,NPW.2,TPW.2,ST1.2,ST2.2,ST3.2,ST4.2,ST5.2
0,B.Becker,A.Murray,1,0,0,3,59,29,41,14,...,10,5,23,17,NaN,6,6,6,NaN,NaN
1,J.Ward,Y-H.Lu,1,0,1,3,62,77,38,35,...,15,2,46,39,NaN,6,6,7,7.0,NaN
2,N.Mahut,J.Hajek,1,1,3,0,72,44,28,10,...,1,0,19,12,NaN,2,4,3,NaN,NaN
3,T.Robredo,A.Bogomolov Jr.,1,1,3,0,77,40,23,12,...,0,0,22,13,NaN,2,2,4,NaN,NaN
4,R.Haase,M.Youzhny,1,0,0,3,68,61,32,15,...,21,3,44,30,NaN,6,7,7,NaN,NaN


12. Ajouter une colonne `Tournoi` dans les dataframes `rg` et `wb` contenant respectivement les chaînes de caractères `"RG"` et `"WB"`.

In [67]:
rg["Tournoi"] = "RG"
wb["Tournoi"] = "WB"

               Player1                  Player2  Round  Result  FNL.1  FNL.2  \
0  Pablo Carreno-Busta            Roger Federer      1       0      0      3   
1     Somdev Devvarman  Daniel Munoz-De La Nava      1       1      3      0   
2         Tobias Kamke            Paolo Lorenzi      1       1      3      2   
3     Julien Benneteau        Ricardas Berankis      1       1      3      1   
4          Lukas Lacko              Sam Querrey      1       0      0      3   

   FSP.1  FSW.1  SSP.1  SSW.1  ...  BPW.2  NPA.2  NPW.2  TPW.2  ST1.2  ST2.2  \
0     62     27     38     11  ...      7     14     18     88      6      6   
1     62     54     38     22  ...     16     22     25    106      3      3   
2     62     53     38     15  ...     18     19     27    139      3      3   
3     72     87     28     19  ...     13     33     43    149      6      3   
4     52     31     48     22  ...      7     12     13     93      6      6   

   ST3.2  ST4.2  ST5.2  Tournoi  
0   

13. Concaténer les deux dataframes dans un nouveau dataframe `tennis`.

In [68]:
tennis = pd.concat([rg, wb], ignore_index=True)
print(tennis[["Player1", "Player2", "Tournoi"]])

                 Player1                  Player2 Tournoi
0    Pablo Carreno-Busta            Roger Federer      RG
1       Somdev Devvarman  Daniel Munoz-De La Nava      RG
2           Tobias Kamke            Paolo Lorenzi      RG
3       Julien Benneteau        Ricardas Berankis      RG
4            Lukas Lacko              Sam Querrey      RG
..                   ...                      ...     ...
234             D.Ferrer              J.Del Potro      WB
235           N.Djokovic                T.Berdych      WB
236           J.Janowicz                 A.Murray      WB
237           N.Djokovic              J.Del Potro      WB
238           N.Djokovic                 A.Murray      WB

[239 rows x 3 columns]


14. Utiliser le dataframe `tennis` pour comparer le nombre moyen d'aces par match à chaque niveau du tournoi à Roland Garros et à Wimbledon. Afficher le résultat en format large.

In [81]:
print(
    (tennis["ACE.1"] + tennis["ACE.2"])
    .to_frame(name="Aces")
    .groupby([tennis.Tournoi, tennis.Round])
    .mean()
    .reset_index()
    .pivot(index="Tournoi", columns="Round", values="Aces")
)

Round           1          2        3       4     5     6     7
Tournoi                                                        
RG       13.47619  13.193548  12.5625   9.125   7.0  10.0   6.0
WB       21.12500  23.869565  24.0000  24.375  26.5  27.5  13.0


15. Quelle différence y a-t-il dans le format des noms des joueurs entre les dataframes `rg` et `wb` ?

In [82]:
# Le prénom des joueurs est limité à son initiale (e.g. "Roger Federer" devient "R.Federer")

print("*** ROLAND GARROS ***")
print(rg[rg.Player2.str.contains("Federer")].Player2)

print("*** WIMBLEDON ***")
print(wb[wb.Player2.str.contains("Federer")].Player2)

*** ROLAND GARROS ***
0      Roger Federer
63     Roger Federer
94     Roger Federer
110    Roger Federer
118    Roger Federer
Name: Player2, dtype: object
*** WIMBLEDON ***
16    R.Federer
71    R.Federer
Name: Player2, dtype: object


16. Construire un dataframe `rg_victoires` avec les trois colonnes suivantes pour le tournoi de Roland Garros :
- `joueur` : nom du joueur tel qu'il est donné dans `rg`,
- `nom_joueur` : nom de famille du joueur uniquement,
- `n_victoire` : nombre de matchs gagnés dans le tournoi.

In [89]:
rg_victoires=(
    pd.concat([rg["Player1"], rg["Player2"]])
        .drop_duplicates()
        .to_frame(name="joueur")
)
rg_victoires["nom_joueur"] = rg_victoires["joueur"].str.split(" ").str[-1]

def n_victoire(df, joueur):
    return (
        len(df[(df.Player1 == joueur) & (df.Result == 1)])
        + len(df[(df.Player2 == joueur) & (df.Result == 0)])
    )

rg_victoires["n_victoire"] = (
    rg_victoires.joueur
    .apply(lambda joueur: n_victoire(rg, joueur))
)

rg_victoires.sort_values(by=["n_victoire","nom_joueur"], ascending=[False,True], inplace=True)

print(rg_victoires)

                 joueur nom_joueur  n_victoire
47         Rafael Nadal      Nadal           7
122        David Ferrer     Ferrer           6
62       Novak Djokovic   Djokovic           5
15   Jo-Wilfried Tsonga     Tsonga           5
0         Roger Federer    Federer           4
..                  ...        ...         ...
42      Maxime Teixeira   Teixeira           0
57        Bernard Tomic      Tomic           0
55          Jiri Vesely     Vesely           0
33       Rhyne Williams   Williams           0
28          Jurgen Zopp       Zopp           0

[127 rows x 3 columns]


17. Construire un dataframe `wb_victoires` avec les trois colonnes suivantes pour le tournoi de Wimbledon :
- `joueur` : nom du joueur tel qu'il est donné dans `wb`,
- `nom_joueur` : nom de famille du joueur uniquement,
- `n_victoire` : nombre de matchs gagnés dans le tournoi.

In [91]:
wb_victoires=(
    pd.concat([wb["Player1"], wb["Player2"]])
        .drop_duplicates()
        .to_frame(name="joueur")
)
wb_victoires["nom_joueur"] = wb_victoires["joueur"].str.split(".").str[-1]

wb_victoires["n_victoire"] = (
    wb_victoires.joueur
    .apply(lambda joueur: n_victoire(wb, joueur))
)

wb_victoires.sort_values(by=["n_victoire","nom_joueur"], ascending=[False,True], inplace=True)

print(wb_victoires)

          joueur nom_joueur  n_victoire
0       A.Murray     Murray           7
63    N.Djokovic   Djokovic           6
32   J.Del Potro  Del Potro           5
48     T.Berdych    Berdych           4
101   J.Janowicz   Janowicz           4
..           ...        ...         ...
28       A.Ungur      Ungur           0
1         J.Ward       Ward           0
24    S.Wawrinka   Wawrinka           0
45    H.Zeballos   Zeballos           0
23        J.Zopp       Zopp           0

[127 rows x 3 columns]


18. Faire une jointure entre `rg_victoires` et `wb_victoires` sur la colonne `nom_joueur` pour comparer le nombre de victoires par tournoi pour chaque joueur. Expliquer la différence de résultat selon que la jointure est à gauche, à droite, intérieure ou extérieure.

In [102]:
print(
    rg_victoires[["nom_joueur", "n_victoire"]].merge(
        wb_victoires[["nom_joueur", "n_victoire"]],
        on="nom_joueur",
        how="left",
        suffixes=("_rg", "_wb")
    )
)

    nom_joueur  n_victoire_rg  n_victoire_wb
0        Nadal              7            0.0
1       Ferrer              6            3.0
2     Djokovic              5            6.0
3       Tsonga              5            1.0
4      Federer              4            1.0
..         ...            ...            ...
124   Teixeira              0            NaN
125      Tomic              0            3.0
126     Vesely              0            NaN
127   Williams              0            NaN
128       Zopp              0            0.0

[129 rows x 3 columns]


In [103]:
print(
    rg_victoires[["nom_joueur", "n_victoire"]].merge(
        wb_victoires[["nom_joueur", "n_victoire"]],
        on="nom_joueur",
        how="right",
        suffixes=("_rg", "_wb")
    )
)

    nom_joueur  n_victoire_rg  n_victoire_wb
0       Murray            NaN              7
1     Djokovic            5.0              6
2    Del Potro            NaN              5
3      Berdych            0.0              4
4     Janowicz            2.0              4
..         ...            ...            ...
125      Ungur            NaN              0
126       Ward            NaN              0
127   Wawrinka            4.0              0
128   Zeballos            1.0              0
129       Zopp            0.0              0

[130 rows x 3 columns]


In [104]:
print(
    rg_victoires[["nom_joueur", "n_victoire"]].merge(
        wb_victoires[["nom_joueur", "n_victoire"]],
        on="nom_joueur",
        how="inner",
        suffixes=("_rg", "_wb")
    )
)

     nom_joueur  n_victoire_rg  n_victoire_wb
0         Nadal              7              0
1        Ferrer              6              3
2      Djokovic              5              6
3        Tsonga              5              1
4       Federer              4              1
..          ...            ...            ...
101  Stakhovsky              0              2
102    Stepanek              0              1
103      Struff              0              2
104       Tomic              0              3
105        Zopp              0              0

[106 rows x 3 columns]


In [105]:
print(
    rg_victoires[["nom_joueur", "n_victoire"]].merge(
        wb_victoires[["nom_joueur", "n_victoire"]],
        on="nom_joueur",
        how="outer",
        suffixes=("_rg", "_wb")
    )
)

    nom_joueur  n_victoire_rg  n_victoire_wb
0        Nadal            7.0            0.0
1       Ferrer            6.0            3.0
2     Djokovic            5.0            6.0
3       Tsonga            5.0            1.0
4      Federer            4.0            1.0
..         ...            ...            ...
148    Odesnik            NaN            0.0
149       Reid            NaN            0.0
150     Rochus            NaN            0.0
151      Ungur            NaN            0.0
152       Ward            NaN            0.0

[153 rows x 3 columns]
